# Jaundice Detection Project

## Skin Detection

In [ ]:
# our custom transformer to detect only skin
import matplotlib.pyplot as plt
import cv2
import numpy as np

def custom_skin_detector(img):
    img = np.array(img)
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #skin color range for h in hsv
    HSV_mask = cv2.inRange(img_HSV[:,:,0], np.array((0)), np.array((17)))
    HSV_mask = cv2.morphologyEx(HSV_mask,cv2.MORPH_OPEN,np.ones((3,3),np.uint8))

    #converting from gbr to YCbCr color space
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    #putting all values of y to 0
    img_YCrCb[:,:,0] = 0

    #skin color range for ycrcb color space 
    YCrCb_mask = cv2.inRange(img_YCrCb, np.array((0, 135, 85)), np.array((255,180,135))) 
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #global mask made from YCrCb mask and hsv mask
    global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
    global_mask=cv2.medianBlur(global_mask,3)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))
    global_img = cv2.bitwise_and(img,img,mask=global_mask)
    global_img = cv2.cvtColor(global_img,cv2.COLOR_BGR2RGB)
    return global_img

## Getting the data

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np

# transformations
transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ToTensor()
])

# Load the datasets
dataset = datasets.ImageFolder('./images',transform=transform)


# combined dataset
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size,test_size],generator=torch.Generator().manual_seed(42))


# Create the dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [3]:
data = list(iter(train_loader))
data[:5]

[[tensor([[[[0.6118, 0.6196, 0.6196,  ..., 0.6745, 0.6745, 0.6745],
            [0.6235, 0.6118, 0.6157,  ..., 0.6745, 0.6745, 0.6745],
            [0.6196, 0.6235, 0.6078,  ..., 0.6784, 0.6745, 0.6745],
            ...,
            [0.7333, 0.7333, 0.7333,  ..., 0.8549, 0.8549, 0.8549],
            [0.7059, 0.7176, 0.7255,  ..., 0.8588, 0.8588, 0.8588],
            [0.6980, 0.7059, 0.7098,  ..., 0.8588, 0.8588, 0.8588]],
  
           [[0.5412, 0.5412, 0.5294,  ..., 0.6353, 0.6353, 0.6353],
            [0.5490, 0.5373, 0.5294,  ..., 0.6353, 0.6353, 0.6353],
            [0.5451, 0.5451, 0.5294,  ..., 0.6392, 0.6353, 0.6353],
            ...,
            [0.6549, 0.6627, 0.6627,  ..., 0.7882, 0.7882, 0.7882],
            [0.6314, 0.6471, 0.6549,  ..., 0.7922, 0.7922, 0.7922],
            [0.6235, 0.6353, 0.6431,  ..., 0.7922, 0.7922, 0.7922]],
  
           [[0.2510, 0.2745, 0.3137,  ..., 0.4980, 0.4980, 0.4980],
            [0.2784, 0.2745, 0.2941,  ..., 0.4980, 0.4980, 0.4980],
      

## Models

In [4]:
# # Defining models

# import torch.nn as nn
# import torch.nn.functional as F
# from torchvision import models

# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 1)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# device = "cuda" if torch.cuda.is_available() else 'cpu'
# model = SimpleCNN()

# # CNN model
# def CNN():
#     model = SimpleCNN()
#     model = model.to(device)

# # VGG model
# def VGG():
#     model = models.vgg16(pretrained= True)
#     model.eval()
#     with torch.no_grad():
#         nn_filters = model.classifier[6].in_features
#         model.classifier[6] = nn.Linear(nn_filters,1)
#         model = model.to(device)

# def resnet():
#     model = models.resnet34(pretrained = True)
#     model.eval()

#     #add a new final layer
#     with torch.no_grad():
#         nr_filters = model.fc.in_features  #number of input features of last layer
#         model.fc = nn.Linear(nr_filters, 1)
#         model = model.to(device)

def alexnet():
    model = models.alexnet(pretrained = True)
    model.eval()
    with torch.no_grad():
        nn_filters = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(nn_filters,1)
        model = model.to(device)

# def densenet():
#     model = models.densenet121(pretrained = True)
#     model.eval()
#     with torch.no_grad():
#         nn_filters = model.classifier.in_features
#         model.classifier = nn.Linear(nn_filters,1)
#         model = model.to(device)

# def googlenet():
#     model = models.googlenet(pretrained = True)
#     model.eval()
#     with torch.no_grad():
#         nn_filters = model.fc.in_features
#         model.fc = nn.Linear(nn_filters,1)
#         model = model.to(device)

def inceptionv3():
    model = models.inception_v3(pretrained = True)
    model.eval()
    with torch.no_grad():
        nn_filters = model.fc.in_features
        model.fc = nn.Linear(nn_filters, 1)
        model = model.to(device)

# def mobilenet():
#     model = models.mobilenet_v2(pretrained = True)
#     model.eval()
#     with torch.no_grad():
#         nn_filters = model.classifier[1].in_features
#         model.classifier[1] = nn.Linear(nn_filters,1)
#         model = model.to(device)

# def squeezenet():
#     model = models.squeezenet1_1(pretrained = True)
#     model.eval()
#     with torch.no_grad():
#         model.classifier[1] = nn.Conv2d(512,1, kernel_size=(1,1))
#         model.num_classes = 1
#         model = model.to(device)

# model_id = 8

# if model_id == 0:
#     CNN()
# elif model_id == 1:
#     VGG()
# elif model_id == 2:
#     resnet()
# elif model_id == 3:
#     alexnet()
# elif model_id == 4:
#     densenet()
# elif model_id == 5:
#     googlenet()
# elif model_id == 6:
#     inceptionv3()
# elif model_id == 7:
#     mobilenet()
# else:
#     squeezenet()

from torchvision import models
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else 'cpu'
model = models.squeezenet1_1(pretrained=True)

model.eval()
with torch.no_grad():
    model.classifier[1] = nn.Conv2d(512, 1, kernel_size=(1,1))
    model.num_classes = 1
    model = model.to(device)

C:\Users\divij\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\divij\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## optimizer and loss functions

In [5]:
optimizer_fn = torch.optim.Adam(model.parameters(),lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()

## Train and Test loops

In [6]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) *100
    return acc

In [7]:
# Training loop
from tqdm import tqdm
for epoch in range(5):
    print(f'!!epoch {epoch} is running!!')

    cum_loss = 0
    for i,(images, labels) in tqdm(enumerate(train_loader),total = len(train_loader)):
        # Forward pass
        model.train()
        images = images.to(device)
        labels = labels.unsqueeze(1).type(torch.float32)
        labels= labels.to(device)

        output_logits = model(images)
        output_pred = torch.round(torch.sigmoid(output_logits))

        loss = loss_fn(labels, output_pred)
        cum_loss += loss
        
        # Backward pass and optimization
        optimizer_fn.zero_grad()
        loss.backward()
        optimizer_fn.step()
    print(f'Epoch {epoch+1}, Train Loss: {cum_loss/len(train_loader)}')

    # test loop
    model.eval()
    with torch.no_grad():
        cum_loss = 0
        cum_acc = 0
        for images_test,labels_test in test_loader:
            images_test = images_test.to(device)
            labels_test = labels_test.unsqueeze(1).type(torch.float32)
            labels_test = labels_test.to(device)

            test_logits = model(images_test)
            test_pred = torch.round(torch.sigmoid(test_logits))

            test_loss = loss_fn(test_pred,labels_test)
            cum_loss += test_loss
            cum_acc += accuracy_fn(y_true = labels_test, y_pred = test_pred)
        print(f'Epoch {epoch+1}, Test Loss {cum_loss/len(test_loader)}, accuracy {cum_acc/len(test_loader)}')

!!epoch 0 is running!!


100%|██████████| 19/19 [02:56<00:00,  9.29s/it]


Epoch 1, Train Loss: 0.41260668635368347
Epoch 1, Test Loss 0.5820116996765137, accuracy 73.125
!!epoch 1 is running!!


 21%|██        | 4/19 [00:32<02:00,  8.06s/it]

In [ ]:
# def make_train_step(model, optimizer, loss_fn):
#   def train_step(x,y):
#     #make prediction
#     yhat = model(x) #forward pass
#     #enter train mode
#     model.train() # make model to train
#     #compute loss
#     loss = loss_fn(yhat,y) # calculate loss

#     loss.backward() # backward propogation
#     optimizer.step() #optimizer step
#     optimizer.zero_grad() 
#     #optimizer.cleargrads()

#     return loss
#   return train_step

In [ ]:
# train_step = make_train_step(model, optimizer_fn, loss_fn)
# from tqdm import tqdm

# losses = []
# val_losses = []

# epoch_train_losses = []
# epoch_test_losses = []


# n_epochs = 2


# early_stopping_tolerance = 0.03
# early_stopping_threshold = 0.003

# for epoch in range(n_epochs):
#   print(f"epoch {epoch} is running")
#   epoch_loss = 0
#   for i ,data in tqdm(enumerate(train_loader), total = len(train_loader)): #iterate ovre batches
#     x_batch , y_batch = data
#     x_batch = x_batch.to(device) #move to gpu
#     y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
#     y_batch = y_batch.to(device)

#     loss = train_step(x_batch, y_batch)
#     epoch_loss += loss/len(train_loader)
#     losses.append(loss)

#   epoch_train_losses.append(epoch_loss)
#   print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))

#   #validation doesnt requires gradient
#   with torch.no_grad():
#     cum_loss = 0
#     for data in test_loader:
#       x_batch,y_batch = data
#       x_batch = x_batch.to(device)
#       y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
#       y_batch = y_batch.to(device)
#       model.eval()
#       yhat = model(x_batch)
#       val_loss = loss_fn(yhat,y_batch)
#       cum_loss += loss/len(test_loader)
#       val_losses.append(val_loss.item())


#     epoch_test_losses.append(cum_loss)
#     print(f'Epoch : {epoch+1}, val loss : {cum_loss}')

#     best_loss = min(epoch_test_losses)

#     #save best model
#     if cum_loss <= best_loss:
#       best_model_wts = model.state_dict()

#     #early stopping
#     early_stopping_counter = 0
#     if cum_loss > best_loss:
#       early_stopping_counter +=1

#     if (early_stopping_counter == early_stopping_tolerance) or (best_loss <= early_stopping_threshold):
#       print("/n Terminating: early stopping")
#       break #terminate training

# #load best model
# model.load_state_dict(best_model_wts)

## Accuracy and plotting